In [25]:
from random import choice
import contracts
import numpy as np
from geometry import translation_angle_from_SE2

contracts.disable_all()
from duckietown_world import SE2Transform
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html

dw.logger.setLevel(50)


First, we load a map.

In [26]:
available_map = dw.list_maps()
print(available_map)
map_name = choice(available_map)
print(f"Loading {map_name}")
m = dw.load_map("robotarium2")
sk = dw.get_skeleton_graph(m)
ipython_draw_html(m);

['udem1', 'regress_4way_drivable', 'loop_empty', 'small_loop', 'regress_4way_adam', 'loop_dyn_duckiebots', 'ETH_small_loop_3', 'straight_road_down', 'field1', 'ETH_large_loop', 'udem1_empty', '4way_bordered', 'experiment_loop', 'calibration_map_ext', 'loop_pedestrians', 'ETH_small_loop_1', 'ETHZ_autolab_technical_track', 'TTIC_ripltown', 'calibration_map_int', 'small_loop_bordered', 'zigzag_dists_bordered', 'ETHZ_loop', 'TTIC_large_loop', 'ETH_small_loop_2', 'robotarium1', 'ETHZ_autolab_fast_track', '4way', 'TTIC_loop', 'ETHZ_loop_bordered', 'small_loop_cw', 'ETHZ_autolab_technical_track_bordered', 'Montreal_loop', 'ETH_intersection_map', 'straight_road', 'robotarium2', 'MOOC_modcon', 'ETH_small_loop_2_bordered', 'loop_obstacles', 'zigzag_dists']
Loading loop_obstacles


INFO:duckietown_world: area: RectangularArea(pmin=[-0.155 -0.151],pmax=[ 6.01499996 11.06117927])
DEBUG:commons.fs:Written 4.2MB to: out/ipython_draw_html/139813293471920/drawing.html
INFO:duckietown_world:Written SVG
 fn_svg: out/ipython_draw_html/139813293471920/drawing.svg


Now let's sample an initial pose.

In [27]:
from duckietown_world.world_duckietown.sampling_poses import sample_good_starting_pose

q = sample_good_starting_pose(m, only_straight=False, along_lane=0.02)
print(SE2Transform.from_SE2(q))

SE2Transform([0.42147296667099, 6.446694374084473],85.9)


We use the `get_lane_pose` function to get the list of lanes in which the Duckiebot could be.

In [28]:
from duckietown_world.world_duckietown import get_lane_poses
import numpy as np

possibilities = list(get_lane_poses(m, q))
print(f"There are {len(possibilities)} possibilities")
glpr = possibilities[0]
from zuper_typing import debug_print

ctr_points = glpr.lane_segment.control_points
abs_ctr_points = []
for ctr_point in ctr_points:
    abs_point = SE2Transform.from_SE2(
        glpr.lane_segment_transform.asmatrix2d().m @ ctr_point.as_SE2())
    abs_ctr_points.append(
        abs_point
    )

print(ctr_points)
print(abs_ctr_points)
ctr_point_id = None
print("looking for a match of ", abs_ctr_points[-1])
for node_id, node in sk.G.nodes.items():
    print("candidate: ", node["point"])
    if np.allclose(node["point"].p, abs_ctr_points[-1].p, rtol=1.e-3):
        ctr_point_id = node_id
        print(node["point"].theta)
        print(abs_ctr_points[-1].theta)
        break

print("ctr_point: ", ctr_point_id)
succ = sk.G[ctr_point_id]



There are 1 possibilities
[SE2Transform([-0.5, -0.22],0.0), SE2Transform([-0.3, -0.3],-45.0), SE2Transform([-0.22, -0.5],-90.0)]
[SE2Transform([0.421199984550476, 6.434999972581863],90.0), SE2Transform([0.46799998283386224, 6.551999968290329],45.0), SE2Transform([0.5849999785423279, 6.598799966573715],-0.0)]
looking for a match of  SE2Transform([0.5849999785423279, 6.598799966573715],-0.0)
candidate:  SE2Transform([1.5911999714374543, 9.944999247789383],90.0)
candidate:  SE2Transform([1.33379998087883, 9.944999247789383],-90.0)
candidate:  SE2Transform([5.6861998808383944, 9.944999247789383],90.0)
candidate:  SE2Transform([5.42879989027977, 9.944999247789383],-90.0)
candidate:  SE2Transform([1.33379998087883, 9.35999920964241],-90.0)
candidate:  SE2Transform([1.754999965429306, 9.523799203634262],-0.0)
candidate:  SE2Transform([1.754999965429306, 9.781199194192887],-180.0)
candidate:  SE2Transform([1.591199971437454, 9.35999920964241],90.0)
candidate:  SE2Transform([5.264999836683273, 

KeyError: None

In [ ]:
# nodes
print(list(sk.G))
# edges
for n1, n2 in sk.G.edges():
    data = sk.G.get_edge_data(n1, n2)
    one_lane = data[0]['lane']
    print(f'I can go from {n1} to {n2} using lane {one_lane}')

#print(sk.root2.children[0])
#print(sk.root2.children)
#print(sk.G.edges)
#print(sk.G0.edges)

